# Mangoes : Composition

This notebook illustrates how to use the composition module of mangoes to create phrase vectors by combining word vectors. 

In this notebook, we will focus on adjective-noun (AN) phrases. 

In [7]:
import mangoes
import mangoes.composition

import sklearn.metrics

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

## Content of this notebook

1. [Get a representation to learn from](#1.-Get-a-representation-to-learn-from)
2. [Compose](#2.-Compose)
3. [Learn the parameters and compose](#3.-Learn-the-parameters-and-compose)


## 1. Get a representation to learn from

Since we want to learn composition parameters from an embedding, we first need to build a representation for adjectives, nouns and AN. 

### Vocabulary
Let's choose a vocabulary from our corpus :

In [8]:
corpus = mangoes.Corpus("data/wiki_article_en", lower= True, ignore_punctuation=True)

Counting words: 0it [00:00, ?it/s]

In [9]:
## Checking out Corpus attributes 
print(corpus.content)
## Check if the given corpus is annotated in Boolean 
print(corpus.annotated)
print(corpus.nb_sentences)
print(corpus.name)
print(corpus.language)
print(corpus.lower)

data/wiki_article_en
False
382
data/wiki_article_en
None
True


In [10]:
adjectives = ['french', 'american', 'spanish', 'russian', 'italian', 'german', 'european', 
              'mexican', 'socialist', 'libertarian', 'marxist', 'first', 'social', 'free', 
              'important', 'different', 'new', 'general', 'great']
nouns = ['anarchist', 'revolution', 'movement', 'school', 'association', 'section', 'revolt']
adj_nouns = ['french revolution', 'mexican revolution', 'spanish revolution', 
             'german revolution', 'russian revolution',
             'german anarchist', 'italian anarchist', 'american anarchist', 
             'spanish anarchist', 'russian anarchist',
             'social revolt', 'social movement', 'social revolution',
             'socialist section', 'italian section']

In [11]:
vocabulary = mangoes.Vocabulary(adjectives + nouns + adj_nouns)

### Create the representation

In [12]:
# choose the 100 most frequent words as context words
context_words = corpus.create_vocabulary(filters=[mangoes.corpus.truncate(100)])
# consider the sentence as context
context = mangoes.context.Sentence(vocabulary=context_words)
# count the cooccurrences 
coocc_count = mangoes.counting.count_cooccurrence(corpus, vocabulary, context)
# apply PPMI to the counts to get a first sparse representation
sparse_representation = mangoes.create_representation(coocc_count,
                                                      weighting=mangoes.weighting.PPMI())
# apply SVD to the weighted counts to get a dense representation
representation = mangoes.create_representation(sparse_representation,
                                               reduction=mangoes.reduction.SVD(dimensions=20))

display(representation.to_df())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
french,9.232378e-01,2.518977e-01,1.976913e-01,5.823899e-01,1.021536e-01,-2.346297e+00,3.608367e-01,-1.178371e+00,-1.025432e+00,2.269521e-01,-1.552892e+00,5.255774e-01,5.269176e-01,-1.348291e+00,2.090963e+00,1.986439e-01,3.327315e-01,1.449723e+00,-5.417322e-01,2.358469e+00
american,1.011921e+00,2.307812e-01,2.674187e-01,9.504112e-01,-6.560689e-01,1.190331e+00,-2.447758e-01,-2.387688e+00,2.246507e+00,-4.227611e-01,1.097492e+00,-1.759385e+00,-1.999450e-01,9.939877e-01,1.766173e-01,1.643640e+00,-1.360573e-01,4.157244e-01,-2.086322e-01,3.515391e+00
spanish,3.844900e-01,-9.919648e-01,7.720455e-01,1.026798e+00,-4.559113e-01,-4.552277e-01,7.956519e-01,3.984610e-01,-7.528701e-01,4.851349e-01,5.279303e-01,-5.622603e-01,-3.753503e-01,9.983573e-01,-5.887961e-01,6.857075e-01,3.892328e-01,6.850663e-03,9.775506e-01,1.396507e+00
russian,7.174436e-03,-7.566456e-01,-1.523177e+00,1.475042e+00,1.545955e+00,7.723306e-01,-6.539087e-02,4.700323e-01,-6.036873e-01,-2.046548e-01,-2.844825e+00,-1.218294e-01,-8.964543e-02,-1.388543e+00,1.002374e+00,2.457476e+00,1.001777e+00,-4.625475e-01,-5.243237e-01,2.551947e+00
italian,-2.207848e-02,7.327243e-01,-8.932005e-01,1.973344e-01,4.093956e-01,-6.570296e-01,-8.338090e-01,-1.060872e-01,4.371266e-01,-2.335336e+00,-8.960702e-01,-1.982598e-01,-3.248204e-01,6.170946e-01,-1.345871e+00,-1.785683e+00,3.152765e+00,-5.528974e-01,1.344408e+00,3.146816e+00
german,9.044918e-02,-6.259619e-01,-2.763328e-01,2.095167e-01,-1.075197e+00,-3.684076e-01,-1.101813e+00,-3.883773e-01,-2.962404e-01,2.034811e+00,-5.165124e-01,-1.010789e+00,9.372634e-01,-1.042798e+00,-3.383304e+00,1.075470e+00,2.720697e-01,3.899385e-01,-1.884152e+00,3.002028e+00
european,-5.195341e-01,-5.981056e-04,5.892556e-01,2.222857e-01,2.540496e+00,7.769772e-01,-2.348189e-03,1.169480e+00,5.883077e-01,2.417877e-01,-3.999941e-01,-8.548218e-01,-4.825368e-01,2.367484e+00,3.601892e-01,-4.478236e-01,-1.478835e+00,3.417065e+00,-2.105688e+00,2.960658e+00
mexican,1.019551e+00,1.232368e+00,-5.785188e-01,2.142487e-01,-3.797900e-02,2.093605e-01,3.066307e-01,7.539880e-01,1.115593e+00,1.917064e+00,-1.144818e+00,4.401947e-01,-1.642987e+00,-6.103295e-01,-3.001053e-01,-1.314326e+00,-2.331189e-01,9.824830e-01,1.500832e+00,1.239624e+00
socialist,1.812040e-01,-3.014550e-01,1.263400e+00,1.836036e-01,-7.090780e-01,1.466179e+00,6.692192e-01,-5.454151e-01,-6.887641e-01,1.085530e+00,1.190846e-01,9.056595e-01,-1.471218e-01,-1.181902e+00,2.723317e+00,-3.553119e-01,6.974363e-01,-9.300267e-01,-9.040450e-01,3.264135e+00
libertarian,-3.673574e-01,3.232677e-01,-1.599944e+00,2.356308e-01,-2.054945e+00,1.699276e+00,1.281625e-01,9.704219e-02,-8.798461e-01,-2.170200e+00,6.833526e-02,6.001506e-01,-1.566836e+00,-3.923883e-01,1.983477e-01,-1.108656e-01,-3.294645e-01,8.986910e-01,4.923463e-01,2.593628e+00


## 2. Compose

In this example, we will derive a new vector $\mathbf{p}$ for the phrase *'spanish revolution'* from the vectors $\mathbf{u}$ and $\mathbf{v}$ representing the words *'spanish'* and *'revolution'*.

Various models are available :


### Additive

In this model, $p$ is obtained by a weighted sum of $u$ and $v$ :

$$\mathbf{p = \alpha u + \beta v}$$

The `mangoes.composition.AdditiveComposer` let you learn $\mathbf{\alpha}$ and $\mathbf{\alpha}$ from your representation and apply them

In [13]:
additive = mangoes.composition.AdditiveComposer(representation)
additive.fit()

print("alpha=", additive.alpha, ", beta=", additive.beta)

alpha= 0.23998432440049766 , beta= 0.18570667435097205


Now we can create a new vector for the phrase "spanish revolution" using these composer :

In [14]:
predicted_vector = additive.predict("spanish", "revolution")

We can measure the distance between this vector and the 'observed' one :

In [15]:
observed_vector = representation[('spanish', 'revolution')]
sklearn.metrics.pairwise_distances(observed_vector.reshape(1,-1), predicted_vector.reshape(1,-1), metric='cosine')

array([[0.25506289]])

And check that the observed vector is (one of) the closet word(s) to the predicted one :

In [16]:
representation.get_closest_words(predicted_vector)

[('spanish', 0.11374605394199377),
 ('revolution', 0.19878176530487057),
 (Bigram(first='spanish', second='revolution'), 0.2550628862532164),
 ('great', 0.4006380255217601),
 ('american', 0.47959369999437507),
 (Bigram(first='social', second='revolt'), 0.4991334812931577),
 (Bigram(first='italian', second='anarchist'), 0.5272597951652993),
 ('revolt', 0.545477326236689),
 ('first', 0.555967046805788),
 ('socialist', 0.5626577557557526)]

In [17]:
representation.get_closest_words(representation["free"])

[('free', 0.0),
 (Bigram(first='social', second='revolt'), 0.38078405645451285),
 (Bigram(first='spanish', second='revolution'), 0.40018786699686204),
 ('revolution', 0.4019392918969279),
 ('important', 0.4038526248912737),
 ('libertarian', 0.4609095858567147),
 ('anarchist', 0.4702513974745852),
 ('school', 0.48370411646330513),
 (Bigram(first='socialist', second='section'), 0.48880976973598966),
 (Bigram(first='american', second='anarchist'), 0.4998128424672039)]

### Dilation

We can now do the same with dilation model where :
$$ \mathbf{p = (u.u)v + (\lambda - 1)(u.v)} $$

In [18]:
dilation = mangoes.composition.DilationComposer(representation)
dilation.fit()
print("lambda =", dilation.lambda_)

lambda = 0.0479736328125


In [19]:
predicted_vector = dilation.predict('spanish', 'revolution')
representation.get_closest_words(predicted_vector)

[('revolution', 0.08883381060292639),
 ('socialist', 0.3877974085450775),
 (Bigram(first='social', second='revolt'), 0.3892323092098494),
 ('free', 0.39044831954995074),
 ('new', 0.418613193096521),
 ('russian', 0.42490693718161976),
 (Bigram(first='socialist', second='section'), 0.4370704922434915),
 ('section', 0.45431084064948757),
 (Bigram(first='spanish', second='revolution'), 0.4592912722438147),
 ('revolt', 0.4728781544830122)]

### Full Additive

This model two weight matrices such as :

$$\mathbf{p = Au + Bv}$$


In [20]:
full_additive = mangoes.composition.FullAdditiveComposer(representation)
full_additive.fit()

In [21]:
predicted_vector = full_additive.predict("spanish", "revolution")
representation.get_closest_words(predicted_vector)

[(Bigram(first='spanish', second='revolution'), 0.0798612203420066),
 (Bigram(first='russian', second='revolution'), 0.1849092152475038),
 ('revolution', 0.3999667609136286),
 ('free', 0.42422954264848434),
 ('spanish', 0.516363951687342),
 (Bigram(first='german', second='revolution'), 0.6007668725140161),
 ('anarchist', 0.612696317569871),
 ('school', 0.6315393529618919),
 ('libertarian', 0.6483982101625264),
 ('important', 0.65414547506035)]

### Lexical Function

Finally, we can also learn a matrix $U$ for each adjective with the "lexical function" model where :
$$ \mathbf{p = Uv} $$

In [22]:
spanish = mangoes.composition.LexicalComposer(representation, "spanish")
spanish.fit()

In [23]:
predicted_vector = spanish.predict('revolution')
representation.get_closest_words(predicted_vector)

[(Bigram(first='spanish', second='revolution'), 0.0),
 (Bigram(first='russian', second='revolution'), 0.2748850980055798),
 ('revolution', 0.2797298800799053),
 ('free', 0.40018786699686204),
 ('spanish', 0.43533042891574325),
 ('libertarian', 0.597037930537175),
 ('anarchist', 0.6433865256501664),
 ('revolt', 0.6611179002823516),
 ('russian', 0.661944569060751),
 ('section', 0.6661411980893858)]